### Python script to connect to IMAP email inbox

In [3]:
# # file: outlook_imap_reader.py

# import imaplib
# import email
# from email.header import decode_header
# import getpass

# def connect_to_outlook_imap():
#     """
#     Simple function to connect to roy@agtpartners.com.sg IMAP inbox.
#     Uses the exact server settings from your Outlook configuration.
#     """
    
#     # Your specific email configuration (from Outlook settings)
#     email_address = "roy@agtpartners.com.sg"
#     imap_server = "mail.agtpartners.com.sg"  # Your actual IMAP server
#     imap_port = 993  # SSL port from your settings
    
#     try:
#         # Get password for your email
#         print("=== AGT Partners Email Reader ===")
#         print(f"Connecting to: {email_address}")
#         print(f"Server: {imap_server}:{imap_port}")
#         password = getpass.getpass("Enter your password: ")  # Hides password input
        
#         # Connect to AGT Partners IMAP server with SSL
#         print("Connecting to AGT Partners IMAP server...")
#         mail = imaplib.IMAP4_SSL(imap_server, imap_port)
        
#         # Login with your credentials
#         print("Logging in...")
#         mail.login(email_address, password)
#         print("✓ Successfully connected to your AGT Partners email!")
        
#         # Select the inbox folder
#         mail.select("inbox")
        
#         # Search for all emails
#         print("\nFetching latest emails...")
#         status, messages = mail.search(None, "ALL")
        
#         # Get list of email IDs
#         email_ids = messages[0].split()
        
#         # Display the 5 most recent emails
#         # recent_emails = email_ids[-5:]  # Get last 5 emails
#         recent_emails = email_ids[:]  # Get last 5 emails
        
#         print(f"\nShowing {len(recent_emails)} most recent emails from {email_address}:\n")
        
#         # Read each email
#         for i, email_id in enumerate(reversed(recent_emails), 1):
#             # Fetch the email
#             status, msg_data = mail.fetch(email_id, "(RFC822)")
            
#             # Parse the email
#             raw_email = msg_data[0][1]
#             email_message = email.message_from_bytes(raw_email)
            
#             # Extract email details
#             subject = decode_header(email_message["Subject"])[0][0]
#             if isinstance(subject, bytes):
#                 subject = subject.decode()
            
#             sender = email_message["From"]
#             date = email_message["Date"]
            
#             # Display email info
#             print(f"Email {i}:")
#             print(f"  Subject: {subject}")
#             print(f"  From: {sender}")
#             print(f"  Date: {date}")
#             print("-" * 50)
        
#         # Close connection
#         mail.close()
#         mail.logout()
#         print("\n✓ Connection closed successfully!")
        
#     except imaplib.IMAP4.error as e:
#         print(f"IMAP Error: {e}")
#         print("Check your password. The server settings match your Outlook configuration.")
        
#     except Exception as e:
#         print(f"Error: {e}")

# if __name__ == "__main__":
#     # Run the email reader for roy@agtpartners.com.sg
#     connect_to_outlook_imap()

In [ ]:
# file: outlook_imap_reader.py

import imaplib
import email
from email.header import decode_header
import getpass

def safe_decode(text, encoding=None):
    """
    Safely decode text with fallback encodings.
    Handles different character encodings commonly found in emails.
    """
    if isinstance(text, str):
        return text
    
    # List of encodings to try (most common for emails)
    encodings_to_try = [
        encoding,  # Use specified encoding first
        'utf-8',
        'latin-1', 
        'windows-1252',
        'iso-8859-1'
    ]
    
    for enc in encodings_to_try:
        if enc is None:
            continue
        try:
            return text.decode(enc)
        except (UnicodeDecodeError, LookupError):
            continue
    
    # If all fail, use 'replace' to avoid errors
    return text.decode('utf-8', errors='replace')

def connect_to_outlook_imap():
    """
    Simple function to connect to roy@agtpartners.com.sg IMAP inbox.
    Now handles different character encodings properly.
    """
    
    # Your specific email configuration
    email_address = "roy@agtpartners.com.sg"
    imap_server = "mail.agtpartners.com.sg"
    imap_port = 993
    
    try:
        # Get password for your email
        print("=== AGT Partners Email Reader ===")
        print(f"Connecting to: {email_address}")
        password = getpass.getpass("Enter your password: ")
        
        # Connect to IMAP server
        print("Connecting to AGT Partners IMAP server...")
        mail = imaplib.IMAP4_SSL(imap_server, imap_port)
        
        # Login
        print("Logging in...")
        mail.login(email_address, password)
        print("✓ Successfully connected!")
        
        # Select inbox
        mail.select("inbox")
        
        # Get emails
        print("\nFetching latest emails...")
        status, messages = mail.search(None, "ALL")
        email_ids = messages[0].split()
        recent_emails = email_ids[-5:]
        # recent_emails = email_ids[:] # this is to get ALL emails
        
        print(f"\nShowing {len(recent_emails)} most recent emails:\n")
        
        # Read each email with proper encoding handling
        for i, email_id in enumerate(reversed(recent_emails), 1):
            try:
                # Fetch the email
                status, msg_data = mail.fetch(email_id, "(RFC822)")
                raw_email = msg_data[0][1]
                email_message = email.message_from_bytes(raw_email)
                
                # Extract subject with proper encoding handling
                subject_header = email_message["Subject"]
                if subject_header:
                    decoded_parts = decode_header(subject_header)
                    subject_parts = []
                    for part, encoding in decoded_parts:
                        subject_parts.append(safe_decode(part, encoding))
                    subject = ''.join(subject_parts)
                else:
                    subject = "(No Subject)"
                
                # Extract other details (these are usually safer)
                sender = email_message["From"] or "(Unknown Sender)"
                date = email_message["Date"] or "(Unknown Date)"
                
                # Display email info
                print(f"Email {i}:")
                print(f"  Subject: {subject}")
                print(f"  From: {sender}")
                print(f"  Date: {date}")
                print("-" * 50)
                
            except Exception as e:
                print(f"Email {i}: Error reading email - {e}")
                print("-" * 50)
        
        # Close connection
        mail.close()
        mail.logout()
        print("\n✓ Connection closed successfully!")
        
    except imaplib.IMAP4.error as e:
        print(f"IMAP Error: {e}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    connect_to_outlook_imap()

=== AGT Partners Email Reader ===
Connecting to: roy@agtpartners.com.sg
Connecting to AGT Partners IMAP server...
Logging in...
✓ Successfully connected!

Fetching latest emails...

Showing 11713 most recent emails:

Email 1:
  Subject: FactSet Quant Support: #9340297FactSet Quant Support:
 #9340297FactSet Quant Support: #9340297RE: FactSet Quant Support:
 #9340297FactSet Quant Support: #9340297
  From: FactSet Quant Support <quantsupport@factset.com>
  Date: Tue, 16 Sep 2025 05:34:01 +0000 (GMT)
--------------------------------------------------
Email 2:
  Subject: SA: US and South Korea remain divided over investment component of
 trade deal:
  From: FactSet Alerts <FactSet_Alerts@factset.com>
  Date: Tue, 16 Sep 2025 05:02:10 +0000
--------------------------------------------------
Email 3:
  Subject: CH/HK: Folangsi Co Ltd (2499 HK): Non-deal roadshow takeaways:
 Platform pivot drives revenue growth amid global expansion  (NOT RATED)
  From: "DBS Group Research" <groupresearch@dbs.

KeyboardInterrupt: 